In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os

In [2]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [429]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO BORROWS FIRST

In [640]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [719]:
oldSiloBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmBorrows.csv'
try:
    oldSiloBorrowsDb = pd.read_csv (siloFileName)
except:
    oldSiloBorrowsDb = pd.DataFrame()
oldSiloBorrowsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,16651434,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,60500.000000,0.996047,59947.550425,2023-02-17 22:39:23,2023-02-17
1,borrow,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,16644592,1676590307,0x96035b45bc0d22f0ed11283a91145fb001c48370,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,0.600000,1674.366016,995.052978,2023-02-16 23:31:47,2023-02-16
2,borrow,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,16641746,1676556011,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,100000.000000,0.996047,100012.487283,2023-02-16 14:00:11,2023-02-16
3,borrow,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,16634870,1676472911,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,3000.000000,0.996047,3023.153053,2023-02-15 14:55:11,2023-02-15
4,borrow,0xd5dc7147192591c7d19856d12e9278928932a5921653...,16627480,1676383655,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,1000.000000,0.996047,992.079025,2023-02-14 14:07:35,2023-02-14
5,borrow,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,16626801,1676375459,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,1.920000,1674.366016,2885.217480,2023-02-14 11:50:59,2023-02-14
6,borrow,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,16626396,1676370563,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,4000.000000,0.996047,3963.627622,2023-02-14 10:29:23,2023-02-14
7,borrow,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,2457.516074,0.996047,2446.717700,2023-02-13 23:17:35,2023-02-13
8,borrow,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,16622907,1676328371,0x46535cbc3fa574c32ea392d4f89c518290597072,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,26.400000,2809.419269,71910.521424,2023-02-13 22:46:11,2023-02-13
9,borrow,0xbcfc05398a874e98a80e823ffb016c119b06c7b07a7c...,16619977,1676292983,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,3090.000000,0.996047,3085.406589,2023-02-13 12:56:23,2023-02-13


In [720]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldSiloBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

1676673558

In [721]:
# 0xc413dd03555f3eb29d834b482d386b2999dc2eb0

In [722]:
##this captures the ENTIRE list of people who borrowed OHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  6  - latest recordID:  0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5ff4b67d962f8a7b60152   0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5ff4b67d962f8a7b60152-239  - borrows DB length:  6  - latest record from:  2023-02-17 14:39:23 2 days ago
Done. Total Records collected:  6


In [723]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])

siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,16651434,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,60500.0,1.001125,59947.550425,2023-02-17 22:39:23,2023-02-17


In [724]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmBorrows_piped.csv file saved, 1  records


In [725]:
totalBorrowedUSD = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of $',round(totalBorrowedUSD,2),  'has been borrowed in different stablecoins over time')

A total of $ 59947.55 has been borrowed in different stablecoins over time


In [726]:
ohmBorrowedSilo = db.query("Select distinct "
                           "transaction_hash "
                           ",transaction_timestamp "
                           ", dateTime "
                           ",transaction_account_id "
                           ", transaction_asset_symbol "
                           ", transactionReadableAmount "
                           ", transaction_amountUSD "
                           "from siloBorrowsDb where lower(transaction_asset_symbol) = 'gohm'").df()

ohmBorrowedSilo.head(10)

,transaction_hash,transaction_timestamp,dateTime,transaction_account_id,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD


In [727]:
totalSiloBorrowedOHM = sum(ohmBorrowedSilo['transactionReadableAmount'])
totalSiloBorrowedOHMinUSD = sum(ohmBorrowedSilo['transaction_amountUSD'])
print('A total of ',round(totalSiloBorrowedOHM,2),  'OHM has been borrowed in different stablecoins over time, roughly USD$',totalSiloBorrowedOHMinUSD)

A total of  0 OHM has been borrowed in different stablecoins over time, roughly USD$ 0


In [728]:
path =file+'/ohmBorrowedSilo.csv'
ohmBorrowedSilo.to_csv(path, index = False)

path =file+'/ohmBorrowedSilo_piped.csv'
ohmBorrowedSilo.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmBorrowedSilo),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/ohmBorrowedSilo_piped.csv file saved, 0  records


********************************************************************
********************************************************************
SILO COLLATERAL DEPOSITS

In [729]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2809.419269,138354.126796,2023-02-17 22:34:23,2023-02-17
1,deposit,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,50.749992,2809.419269,145833.530065,2023-02-16 13:58:59,2023-02-16
2,deposit,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,16627499,1676383883,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.180346,2809.419269,490.375269,2023-02-14 14:11:23,2023-02-14
3,deposit,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,16627469,1676383523,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.663108,2809.419269,1803.040025,2023-02-14 14:05:23,2023-02-14
4,deposit,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.892669,2809.419269,2431.527276,2023-02-13 23:18:59,2023-02-13
5,deposit,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.087799,2809.419269,2963.036715,2023-02-13 23:14:47,2023-02-13
6,deposit,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,16619969,1676292887,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.812154,2809.419269,4846.273689,2023-02-13 12:54:47,2023-02-13
7,deposit,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2809.419269,363.447391,2023-02-10 22:58:11,2023-02-10
8,deposit,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,9.990000,2809.419269,26839.810526,2023-02-09 10:34:59,2023-02-09
9,deposit,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,16589368,1675923587,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,119.000000,2809.419269,315110.337935,2023-02-09 06:19:47,2023-02-09


In [734]:
oldSiloDepositsDb['transaction_timestamp'].max()

1676673263

In [738]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositsDb['transaction_timestamp'].max()))-5
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)-1
maxSiloDepositsTimestamp

1676673257

In [757]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  6  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  6  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  1 Lines skipped:  1000 records collected:  0  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  6  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  2 Lines skipped:  2000 records collected:  0  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  6  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  3 Lines skipped:  3000 records collected:  0  - latest recordID:  0xecd6140bbb896413a25d34

IndexError: index 6 is out of bounds for axis 0 with size 6

In [758]:
siloDepositsDb.head(10)

,deposits_id,deposits_hash,deposits_blockNumber,deposits_timestamp,deposits_account_id,deposits_market_id,deposits_market_protocol_id,deposits_market_rates_rate,deposits_market_rates_duration,deposits_market_rates_side,deposits_market_rates_type,deposits_market_rates_token_id,deposits_asset_name,deposits_asset_symbol,deposits_asset_decimals,deposits_asset_lastPriceUSD,deposits_amount,deposits_amountUSD
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796
1,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796
2,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796
3,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796
4,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.117764,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796
5,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.614000,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2809.009098,48922199130466954750,138354.126796


In [759]:
siloDepositsDb['event'] ='deposit'
siloDepositsDb['depositsReadableAmount'] = siloDepositsDb['deposits_amount']/(10**(siloDepositsDb['deposits_asset_decimals']))
siloDepositsDb['depositsReadableAmount']=siloDepositsDb['depositsReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['deposits_timestamp'],unit='s'))


#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['deposits_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['deposits_asset_symbol']!='gOHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','deposits_hash','deposits_blockNumber','deposits_timestamp','deposits_account_id','deposits_market_id','deposits_asset_name','deposits_asset_symbol', 'depositsReadableAmount','deposits_asset_lastPriceUSD','deposits_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','deposits_hash','deposits_blockNumber','deposits_timestamp','deposits_account_id','deposits_market_id','deposits_asset_name','deposits_asset_symbol', 'depositsReadableAmount','deposits_asset_lastPriceUSD','deposits_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['deposits_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(1000)

,event,deposits_hash,deposits_blockNumber,deposits_timestamp,deposits_account_id,deposits_market_id,deposits_asset_name,deposits_asset_symbol,depositsReadableAmount,deposits_asset_lastPriceUSD,deposits_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2809.009098,138354.126796,2023-02-17 22:34:23,2023-02-17


In [760]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmDeposits_piped.csv file saved, 1  records


In [764]:
totalDeposited = sum(siloDepositEventsDbTimedClean['depositsReadableAmount'])
totalUSDDeposited = sum(siloDepositEventsDbTimedClean['deposits_amountUSD'])
print('There has been a total of',round(totalDeposited,2),  'gOHM deposited cumulatively as collateral  to date, about USD$',totalUSDDeposited)

There has been a total of 48.92 gOHM deposited cumulatively as collateral  to date, about USD$ 138354.1267963194


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [768]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,withdraw,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,25.000039,2809.419269,71148.084004,2023-02-15 15:27:47,2023-02-15
1,withdraw,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,64.393035,2809.419269,183257.358356,2023-02-15 14:49:59,2023-02-15
2,withdraw,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,36.000000,2809.419269,100991.276713,2023-02-14 16:23:59,2023-02-14
3,withdraw,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,16627341,1676381987,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.883213,2809.419269,2401.521741,2023-02-14 13:39:47,2023-02-14
4,withdraw,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,16601676,1676072123,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2809.419269,363.447391,2023-02-10 23:35:23,2023-02-10
5,withdraw,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,16589407,1675924055,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,19.607121,2809.419269,51919.381911,2023-02-09 06:27:35,2023-02-09
6,withdraw,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,16574012,1675737947,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.194798,2809.419269,510.055435,2023-02-07 02:45:47,2023-02-07
7,withdraw,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,16483146,1674641375,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,20.000000,2809.419269,51345.559733,2023-01-25 10:09:35,2023-01-25
8,withdraw,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,16366146,1673230019,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,2.252766,2809.419269,5812.081890,2023-01-09 02:06:59,2023-01-09
9,withdraw,0x7c24d04318da39f08c7ba4f8d413a487133ce1b1f117...,16242703,1671741731,0x96035b45bc0d22f0ed11283a91145fb001c48370,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,13.520668,2809.419269,34629.263098,2022-12-22 20:42:11,2022-12-22


In [773]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawsDb['transaction_timestamp'].max()))-1
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

1676474866

In [775]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  12  - latest recordID:  0x6874260982e867567e9455254ce574aba7f95db4d6f82bc2485bd1b40a71a83e   0x6874260982e867567e9455254ce574aba7f95db4d6f82bc2485bd1b40a71a83e-347  - withdraws DB length:  12  - latest record from:  2023-02-15 14:58:59 4 days ago
Done. Total Records collected:  12


In [776]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
6,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2809.009098,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
7,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2809.009098,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
8,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2809.009098,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47


In [777]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmWithdraws_piped.csv file saved, 1  records


In [778]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 25.0 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 71148.08


In [779]:
collateralBalance = totalDeposited -(totalWithdrawed)
print(totalDeposited, ' - ',totalWithdrawed,'=', collateralBalance, 'currently deposited as collateral in Silo' )

48.92219913046696  -  25.000038820033232 = 23.922160310433725 currently deposited as collateral in Silo


********************************************************************
********************************************************************
VESTA BORROWS - Their API is poo RN

In [ ]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/shinitakunai/vestafinance-v1_2')

In [ ]:
oldVestaBorrowsDb = pd.DataFrame()
siloFileName = file+'/vesta_borrow_events.csv'
try:
    oldVestaBorrowsDb = pd.read_csv (siloFileName)
except:
    oldVestaBorrowsDb = pd.DataFrame()
oldVestaBorrowsDb.head(100)

In [ ]:
maxVestaBorrowTimestamp = 0
try:
    maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
    maxVestaTroveChangeId = int(maxVestaTroveChangeId)
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
        )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])
except:
    maxVestaTroveChangeId = 0
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
               #,'id_gte': maxVestaTroveChangeId
         }
    )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])

vestaBorrowslist.head(10)

vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
    )
vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.transaction.id
        ,vestaBorrows.transaction.blockNumber
        ,vestaBorrows.debtChange
        ,vestaBorrows.transaction.timestamp
        ,vestaBorrows.trove.owner.id
        ,vestaBorrows.trove.asset.id
        ,vestaBorrows.trove.asset.name
    ])

vestaBorrowslist.head(10)

In [ ]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    try:
        maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
        maxVestaTroveChangeId = int(maxVestaTroveChangeId)
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                    ,'id_gte': maxVestaTroveChangeId
             }
            )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])
    except:
        maxVestaTroveChangeId = 0
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                   #,'id_gte': maxVestaTroveChangeId
             }
        )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

In [ ]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(troveChanges_transaction_timestamp::int) dateTime "
               "From vestaBorrowsDb "
                               "order by troveChanges_transaction_timestamp desc"
               ).df()

vestaBorrowsDbTimed.head(1000)

In [ ]:
vestaBorrowsDbTimed = pd.concat([oldVestaBorrowsDb,vestaBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
vestaBorrowsDbTimed.drop_duplicates()
vestaBorrowsDbTimed = db.query("select "
               "    * "
               "From vestaBorrowsDbTimed "
                         "group by 1,2,3,4,5,6,7,8,9,10 "
                     # "order by borrows_amountUSD desc "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

In [ ]:
path =file+'/vesta_borrow_events.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(vestaBorrowsDbTimed),' records')

************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [846]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
#frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [856]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_borrows.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

""


In [848]:
maxBorrowFraxTimestamp = 0
try:
    maxBorrowFraxTimestamp = (max(1,oldFraxActions['actions_timestamp'].max()))-1
except:
        maxBorrowFraxTimestamp = 0
maxBorrowFraxTimestamp = int(maxBorrowFraxTimestamp)
maxBorrowFraxTimestamp

0

In [877]:
actionsQuery = frax.Query.actions(
     orderBy='timestamp'
    ,orderDirection='desc'
    ,first=1000
    #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             ,'timestamp_gte': maxBorrowFraxTimestamp
             }
    #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
#{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

)
actions= sg.query_df([
    actionsQuery.id
    ,actionsQuery.type
    ,actionsQuery.amount
    ,actionsQuery.timestamp
    ,actionsQuery.block
    ,actionsQuery.token.name
    ,actionsQuery.root.address
    ,actionsQuery.pair.asset.symbol
    ,actionsQuery.pair.asset.decimals
    ,actionsQuery.pair.collateral.symbol
    ,actionsQuery.pair.collateral.decimals
    ]
)
actions['dateTime'] =(pd.to_datetime(actions['actions_timestamp'],unit='s'))
actions.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,dateTime
0,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,borrowAsset,0,1676866727,16667337,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47
1,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,addCollateral,7006979439076347667,1676866727,16667337,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47
2,0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee44...,borrowAsset,10000000000000000000000,1676865431,16667229,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 03:57:11
3,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,borrowAsset,0,1676856395,16666484,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35
4,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x2dc9bc4f7c7ac1708768d1f6f6e6a41b8fb4964fd894...,borrowAsset,0,1675973579,16593503,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 20:12:59
96,0x2dc9bc4f7c7ac1708768d1f6f6e6a41b8fb4964fd894...,addCollateral,1849694947953999686,1675973579,16593503,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 20:12:59
97,0x5dea5a1baef298676a0c47666ca76db0d00637a96750...,borrowAsset,5000000000000000000000,1675972547,16593417,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 19:55:47
98,0x2c4738e7f57d0ef25cef1541393689e07f7f01b8774a...,borrowAsset,0,1675971107,16593299,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 19:31:47


In [871]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
                 ,'timestamp_gte': maxBorrowFraxTimestamp
                 }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ,actionsQuery.pair.asset.symbol
        ,actionsQuery.pair.asset.decimals
        ,actionsQuery.pair.collateral.symbol
        ,actionsQuery.pair.collateral.decimals
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionListLength<1000: exit =True
    actionTicker = actionTicker+1
actionsDb['DateTime']=(pd.to_datetime(actionsDb['actions_timestamp'],unit='s'))
print("Done. Total Records collected: ", actionsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  borrowAsset   0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee444981120505b2e49f050d  - actions DB length:  1000  - latest record from:  2023-02-19 19:57:11 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  1000  - latest recordID:  borrowAsset   0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb9956dd84e7206633b56fd4+1  - actions DB length:  2000  - latest record from:  2023-02-19 17:26:35 0 days ago
iterations:  2 Lines skipped:  2000 records collected:  1000  - latest recordID:  addCollateral   0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb9956dd84e7206633b56fd4  - actions DB length:  3000  - latest record from:  2023-02-19 17:26:35 0 days ago
iterations:  3 Lines skipped:  3000 records collected:  1000  - latest recordID:  borrowAsset   0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45fed6f9ad8f4a70acc7def+1  - actions DB length:  4000  - latest record from:  2023-02-19 12:16:23 0 days ago
iterations:  

KeyboardInterrupt: 

In [ ]:
actionsDb['transactionReadableAmount'] = actionsDb['actions_amount']/(10**(actionsDb['actions_pair_asset_decimals']))
actionsDb['transactionReadableAmount']=actionsDb['transactionReadableAmount'].astype('float')
actionsDb['dateTime'] =(pd.to_datetime(actionsDb['actions_timestamp'],unit='s'))
actionsDb.head(2000)

In [ ]:
fraxActionsDb= pd.DataFrame()
fraxActionsDb =pd.concat([actionsDb, oldFraxActions])
fraxActionsDb.head(1000)

In [ ]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )


In [786]:
#we might as well create all the DBs here and save ourselves time and grief
fraxBorrowsDbTimed = db.query("Select distinct * from actionsDb where actions_type = 'borrowAsset' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()


fraxDepositsDbTimed =fraxBorrows = db.query("Select distinct * from actionsDb where actions_type = 'addCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()


fraxWithdrawsDbTimed = db.query("Select distinct * from actionsDb where actions_type = 'removeCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

In [787]:
fraxBorrowsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,transactionReadableAmount,dateTime
0,0x15f93b52199fd785e5e2423777b99e80b93b5ac0ea6f...,borrowAsset,400000000000000000000,1676823143,16663746,Frax,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,FRAX,18,gOHM,18,400.0,2023-02-19 16:12:23


In [788]:
fraxDepositsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,transactionReadableAmount,dateTime
0,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,6.48,2023-02-20 01:26:35
1,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,addCollateral,20000000000000000000,1676837783,16664949,Governance OHM,0x343ee80f82440a498e48ae186e066809750a055b,FRAX,18,gOHM,18,20.00,2023-02-19 20:16:23


In [789]:
fraxWithdrawsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,transactionReadableAmount,dateTime


In [790]:
fraxActionsDbTimed = pd.concat([oldFraxBorrows,fraxBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
fraxActionsDbTimed.sort_values(['actions_id','actions_type','actions_amount','actions_timestamp','actions_block','actions_token_name','actions_root_address','dateTime'])
fraxActionsDbTimed.drop_duplicates(keep='first' )
fraxActionsDbTimed.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,transactionReadableAmount,dateTime
0,0x08cef0dcfa82fb386ec9948dadfaa9dba0da206c6523...,borrowAsset,60000000000000000000000,1671302279,16206258,Frax,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,18,gOHM,18,60000.0,2022-12-17 18:37:59
1,0x52a22509d8397aec4b08a262c8e50ce0e9e810cc1981...,borrowAsset,10000000000000000000000,1671284591,16204790,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,10000.0,2022-12-17 13:43:11
2,0x70031de3f701a9ac2d57f53fc400f51a6918cc09a037...,borrowAsset,12400000000000000000000,1671223127,16199693,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,12400.0,2022-12-16 20:38:47
3,0x91ce97ab7feb2244dbbcc49a2bc94e8859c767eba7eb...,borrowAsset,15000000000000000000000,1671213215,16198877,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,15000.0,2022-12-16 17:53:35
4,0x0622d462adf7b5db61999e4818580e948de56acee446...,borrowAsset,20000000000000000000000,1671212627,16198828,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,20000.0,2022-12-16 17:43:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x2e0270b9a66c3418c22627e1c7de4e9e901517e297d4...,borrowAsset,5000000000000000000000,1673961911,16426804,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,5000.0,2023-01-17 13:25:11
96,0xeb8dea4c2dcc1f9a4bf14020ecd813d5f9e3123fb041...,borrowAsset,13000000000000000000000,1673924123,16423661,Frax,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,FRAX,18,gOHM,18,13000.0,2023-01-17 02:55:23
97,0xbc0973c9735faee148da1ce4e23af88146f32ce35071...,borrowAsset,20000000000000000000000,1673903867,16421986,Frax,0x783f3cad39b6528e192722527429bbbdaad01648,FRAX,18,gOHM,18,20000.0,2023-01-16 21:17:47
98,0xd14c45bcd7f7a533e9fa2a63ef767997212348e43d41...,borrowAsset,5000000000000000000000,1673899283,16421607,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,5000.0,2023-01-16 20:01:23


In [791]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 162  records at 2023-02-19 18:43:04.849564


Collateral check

In [792]:
totalgOHMDepositedFrax = sum(fraxDepositsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMDepositedFrax,2),  'gOHM deposited cumulatively as collateral  to date')

There has been a total of 26.48 gOHM deposited cumulatively as collateral  to date


In [793]:
totalgOHMWithdrawnFrax = sum(fraxWithdrawsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMWithdrawnFrax,2),  'gOHM withdrawn cumulatively as collateral  to date')

There has been a total of 0 gOHM withdrawn cumulatively as collateral  to date


In [794]:
collateralBalance = totalgOHMDepositedFrax-totalgOHMWithdrawnFrax
print(totalgOHMDepositedFrax, ' - ',totalgOHMWithdrawnFrax,'=', collateralBalance, 'currently deposited as collateral in Silo' )

26.48  -  0 = 26.48 currently deposited as collateral in Silo


**********************************************************
MERGE DIFFERENT DBs

In [795]:
#COLLATERALDEPOSITS
fraxDepositsDbPreMerge = db.query(
    "select distinct"
    " actions_id "
    ", actions_timestamp "
    ", dateTime "
    ", actions_root_address deposit_from "
    ", actions_pair_collateral_symbol asset_symbol"
    ", transactionReadableAmount ReadableAmount "
    "from fraxDepositsDbTimed "
).df()
fraxDepositsDbPreMerge['protocol'] = 'Frax'
fraxDepositsDbPreMerge['action'] = 'depositCollateral'
fraxDepositsDbPreMerge.head(10)



,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,1676856395,2023-02-20 01:26:35,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,6.48,Frax,depositCollateral
1,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,1676837783,2023-02-19 20:16:23,0x343ee80f82440a498e48ae186e066809750a055b,gOHM,20.00,Frax,depositCollateral


In [797]:
siloDepositEventsDbTimedClean.head(1)

,event,deposits_hash,deposits_blockNumber,deposits_timestamp,deposits_account_id,deposits_market_id,deposits_asset_name,deposits_asset_symbol,depositsReadableAmount,deposits_asset_lastPriceUSD,deposits_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2809.009098,138354.126796,2023-02-17 22:34:23,2023-02-17


In [798]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " deposits_hash actions_id "
    ", deposits_timestamp actions_timestamp "
    ", dateTime "
    ", deposits_account_id deposit_from "
    ", deposits_asset_symbol asset_symbol "
    ", depositsReadableAmount ReadableAmount "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral


DONT FORGET VESTA WHEN IT GETS FIXED

In [799]:
gOHMCollateralDeposits = pd.concat([siloDepositsDbPreMerge,fraxDepositsDbPreMerge])
gOHMCollateralDeposits.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
0,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,1676856395,2023-02-20 01:26:35,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,6.480000,Frax,depositCollateral
1,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,1676837783,2023-02-19 20:16:23,0x343ee80f82440a498e48ae186e066809750a055b,gOHM,20.000000,Frax,depositCollateral


In [800]:
gOHMCollateralDepositsLength = len(gOHMCollateralDeposits)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/gOHMCollateralDeposits_piped.csv'
gOHMCollateralDeposits.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralDepositsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/gOHMCollateralDeposits_piped.csv file saved, 3  records at 2023-02-19 18:44:41.139067


In [ ]:
#ALL WITHDRAW EVENTS

In [801]:
fraxWithdrawsDbPreMerge = db.query(
    "select distinct"
    " actions_id "
    ", actions_timestamp "
    ", dateTime "
    ", actions_root_address deposit_from "
    ", actions_pair_collateral_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxWithdrawsDbTimed "
).df()
fraxWithdrawsDbPreMerge['protocol'] = 'Frax'
fraxWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
fraxWithdrawsDbPreMerge.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action


In [803]:
siloWithdrawsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash actions_id "
    ", transaction_timestamp actions_timestamp "
    ", dateTime "
    ", transaction_account_id deposit_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloWithdrawEventsDbTimedClean "
).df()
siloWithdrawsDbPreMerge['protocol'] = 'silo'
siloWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
siloWithdrawsDbPreMerge.head(10)


,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral


DONT FORGET VESTA

In [804]:
gOHMCollateralWithdrawals = pd.concat([siloWithdrawsDbPreMerge,fraxWithdrawsDbPreMerge])
gOHMCollateralWithdrawals.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral


In [805]:
gOHMCollateralWithdrawalsLength = len(gOHMCollateralWithdrawals)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False)

path =file+'/gOHMCollateralWithdrawals_piped.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralWithdrawalsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/gOHMCollateralWithdrawals_piped.csv file saved, 1  records at 2023-02-19 18:45:56.563588


In [806]:
#BORROWS

In [807]:
fraxBorrowsDbPreMerge = db.query(
    "select distinct"
    " actions_id "
    ", actions_timestamp "
    ", dateTime "
    ", actions_root_address deposit_from "
    ", 'FRAX' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxBorrowsDbTimed "
).df()
fraxBorrowsDbPreMerge['protocol'] = 'Frax'
fraxBorrowsDbPreMerge['action'] = 'borrowStablecoin'
fraxBorrowsDbPreMerge.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0x15f93b52199fd785e5e2423777b99e80b93b5ac0ea6f...,1676823143,2023-02-19 16:12:23,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,FRAX,400.0,Frax,borrowStablecoin


In [808]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash actions_id "
    ", transaction_timestamp actions_timestamp "
    ", dateTime "
    ", transaction_account_id deposit_from "
    ", 'XAI' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['protocol'] = 'silo'
siloBorrowsDbPreMerge['action'] = 'borrowStablecoin'
siloBorrowsDbPreMerge.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.0,silo,borrowStablecoin


DONT FORGET VESTA

In [809]:
stablecoinsBorrowedWithGOHMCollateral = pd.concat([siloBorrowsDbPreMerge,fraxBorrowsDbPreMerge])
stablecoinsBorrowedWithGOHMCollateral.head(10)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.0,silo,borrowStablecoin
0,0x15f93b52199fd785e5e2423777b99e80b93b5ac0ea6f...,1676823143,2023-02-19 16:12:23,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,FRAX,400.0,Frax,borrowStablecoin


In [810]:
stablecoinsBorrowedWithGOHMCollateralDepositsLength = len(stablecoinsBorrowedWithGOHMCollateralDeposits)
path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits.csv'
stablecoinsBorrowedWithGOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv'
stablecoinsBorrowedWithGOHMCollateralDeposits.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxBorrowsWithGOHMCollateralLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv file saved, 275  records at 2023-02-19 18:46:04.954749


**********************************************************
MERGE EVERYTHING


In [811]:
stablecoinsBorrowedWithGOHMCollateralDeposits.head(1)

,actions_id,actions_timestamp,deposit_from,asset_symbol,depositReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.0,silo,borrowStablecoin


In [812]:
stablecoinsBorrowedWithGOHMCollateralEvents = pd.DataFrame()
stablecoinsBorrowedWithGOHMCollateralEvents = pd.concat([gOHMCollateralDeposits,gOHMCollateralWithdrawals,stablecoinsBorrowedWithGOHMCollateral])
stablecoinsBorrowedWithGOHMCollateralEvents.head(1000)

,actions_id,actions_timestamp,dateTime,deposit_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
0,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,1676856395,2023-02-20 01:26:35,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,6.480000,Frax,depositCollateral
1,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,1676837783,2023-02-19 20:16:23,0x343ee80f82440a498e48ae186e066809750a055b,gOHM,20.000000,Frax,depositCollateral
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.000000,silo,borrowStablecoin
0,0x15f93b52199fd785e5e2423777b99e80b93b5ac0ea6f...,1676823143,2023-02-19 16:12:23,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,FRAX,400.000000,Frax,borrowStablecoin


In [813]:
stablecoinsBorrowedWithGOHMCollateralEventsLength = len(stablecoinsBorrowedWithGOHMCollateralEvents)
path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralEventsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv file saved, 6  records at 2023-02-19 18:46:16.821414


FRAX DAILY SNAPS

In [883]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.pair.collateral.symbol
    ,pairsQuery.pair.collateral.decimals
    ,pairsQuery.pair.asset.symbol
    ,pairsQuery.pair.asset.decimals
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  84  - latest recordID:  gOHM   0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent  - pairSnapshots DB length:  84  - latest record from:  1969-12-31 16:00:00 19408 days ago
Done. Total Records collected:  84


In [884]:
pairSnapshotsDb.head(10)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_collateral_id,pairPerDays_pair_oracleMultiplyAddress_id,pairPerDays_pair_oracleDivideAddress_id,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,352708931626223,5035608515813821703875422,4966638858282894118426824,3049199062322648903525,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,352708931626223,5035608515813821703875422,4966638858282894118426824,3049199062322648903525,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19405,gOHM,18,FRAX,18,0,350112376599804,4698486647811909309719977,4636381316961784483066425,2938561823965620072232,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
5,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19404,gOHM,18,FRAX,18,0,363785846961333,4737561509933966578859758,4675953474727265989294250,2926144943608541855178,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
6,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19403,gOHM,18,FRAX,18,0,346065938064754,4696443773732938768853377,4636381316961784483066425,2926505753609477928993,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
7,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19402,gOHM,18,FRAX,18,0,355065538289602,4675315360621293243183076,4616545966840300870160156,2962449892162877274570,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
8,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,gOHM,18,FRAX,18,0,361722603166047,5214478754084957077245512,5149945278165210104338772,2920785897200163201938,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
9,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19400,gOHM,

In [887]:
pairSnapshotsDb['pairPerDays_totalCollateralReadable'] = pairSnapshotsDb['pairPerDays_totalCollateral']/(10**(pairSnapshotsDb['pairPerDays_pair_collateral_decimals']))
pairSnapshotsDb['pairPerDays_totalCollateralReadable']=pairSnapshotsDb['pairPerDays_totalCollateralReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'] = pairSnapshotsDb['pairPerDays_totalAssetAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalAssetAmountReadable']=pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'].astype('float')


pairSnapshotsDb['dateTime'] =(pd.to_datetime(pairSnapshotsDb['pairPerDays_timestamp'],unit='s'))

pairSnapshotsDb.drop_duplicates(keep ="first",inplace = True)


pairSnapshotsDb.head(2000)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive,pairPerDays_totalCollateralReadable,pairPerDays_totalAssetAmountReadable,dateTime
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,352708931626223,5035608515813821703875422,4966638858282894118426824,3049199062322648903525,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,3049.199062,5.035609e+06,2023-02-20 04:18:47
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,352708931626223,5035608515813821703875422,4966638858282894118426824,3049199062322648903525,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,3049.199062,5.035609e+06,2023-02-20 04:18:47
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,3035.712083,5.035386e+06,2023-02-19 20:16:23
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,3015.712083,5.034722e+06,2023-02-18 22:49:59
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19405,gOHM,18,FRAX,18,0,350112376599804,4698486647811909309719977,4636381316961784483066425,2938561823965620072232,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2938.561824,4.698487e+06,2023-02-17 23:41:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,gOHM,18,FRAX,18,0,416982985435782,3000265799749067769553552,2999681610314728238726349,1702267776392927486576,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,1702.267776,3.000266e+06,2022-12-04 23:12:59
80,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,gOHM,18,FRAX,18,0,435238970363534,3000407417762830870845360,2999931610314728238726349,1460557767872656691963,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,1460.557768,3.000407e+06,2022-12-03 23:30:59
81,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,gOHM,18,FRAX,18,0,424429606966213,3000316417810236657492429,2999931610314728238726349,1298896246642712535146,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,1298.896247,3.000316e+06,2022-12-02 23:55:59
82,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,gOHM,18,FRAX,18,0,455232217741157,1000250018374254924219488,1000177511746822029696096,116544304370705257,...,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,0.116544,1.000250e+06,2022-12-01 21:30:47


In [888]:
fraxPairSnapshotsLength = len(pairSnapshotsDb)
path =file+'/fraxPairSnapshotsLength.csv'
pairSnapshotsDb.to_csv(path, index = False)

path =file+'/fraxPairSnapshotsLength_piped.csv'
pairSnapshotsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxPairSnapshotsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/fraxPairSnapshotsLength_piped.csv file saved, 84  records at 2023-02-19 21:09:43.990581
